# Computer Vision

## Assignment - UNet

### Install

In [1]:
%pip install segmentation_models --quiet
%pip install comet_ml --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Comet Config

In [2]:
from comet_ml import Experiment

experiment = Experiment(project_name = "cv-deep")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/mamello-justice/cv-deep/83761ba811af4689bea27e4e58de9884



### Imports

In [5]:
import time
import os
from os import path

import numpy as np

import tensorflow as tf
import keras
from keras import callbacks
from keras.models import load_model
from segmentation_models import Unet

from cv_gmm_deep.datasets import load_data
from cv_gmm_deep.common import split_data, preprocess_data

print(f"numpy=={np.__version__}")
print(f"tensorflow=={tf.__version__}")
print(f"keras=={keras.__version__}")

ModuleNotFoundError: No module named 'cv_gmm_deep'

### Common

### Constants

In [ ]:
BACKBONE = 'vgg16'

assets_dir = './assets'
data_dir = './data'

default_input_size = [768, 1024]
default_input_dir = path.join(
    assets_dir,
    f'puzzle_corners_{default_input_size[1]}x{default_input_size[0]}')

time_now = int(time.time())

default_cp_path = path.join(data_dir, str(time_now), 'cp.ckpt')
default_model_path = path.join(data_dir, str(time_now), 'model.h5')

### Params

In [ ]:
params = {
    'augmented': True,
    'batch_size': 2,
    'epochs': 2,
    'input_size': default_input_size,
}

experiment.log_parameters(params)

### Setup

In [ ]:
def setup_args():
    return {
        'cp_path': default_cp_path,
        'input_dir': default_input_dir,
        'model_path': default_model_path,
        'update_model': None,
        'cpu': True
    }

### Train

In [ ]:
args = setup_args()

use_cpu = args['cpu']
if use_cpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

augmented = params['augmented']
batch_size = params['batch_size']
epochs = params['epochs']
input_size = params['input_size']

cp_path = args['cp_path']
input_dir = args['input_dir']
model_path = args['model_path']
update_model = args['update_model']

height, width = input_size

raw_x, raw_y = load_data(input_dir, height, width)


In [ ]:
data_x, data_y = preprocess_data(raw_x, raw_y)

In [ ]:
(train_x, train_y), (val_x, val_y), (test_x, test_y) =\
    split_data(data_x, data_y)
    
experiment.log_dataset_hash(train_x)

In [ ]:
checkpoint_cb = callbacks.ModelCheckpoint(filepath=cp_path,
                                          save_weights_only=True,
                                          verbose=1)

if update_model and model_path:
    model = load_model(model_path)
else:
    model = Unet(BACKBONE,
                 encoder_weights='imagenet',
                 input_shape=(*input_size, 3))
    
assert model is not None, "Could not load model"

model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(val_x, val_y),
          callbacks=[checkpoint_cb])

model.save(model_path)

### Comet Commit

In [ ]:
experiment.end()